In [1]:
# query Terrascope archive using terracatalgoueclient
from terracatalogueclient import Catalogue
import datetime as dt

# processes only VV

# define name of result
name = "testRun_01"

catalogue = Catalogue()

In [25]:
# give one date, find closest acqusition before and after
start = dt.date(2020, 10, 1) # input
end = dt.date(2020, 10, 13)

print("[CATALOGUE]: starting search for scenes...")

# first search
cat = catalogue.get_products(
	"urn:eop:VITO:CGS_S1_SLC_L1",
	start = start,
	end = end)

s1a = []
s1b = []

for p in cat:
    path = p.data[0].href 
    iw_index = path.index("IW")
    vm_path = "/data/MTDA/CGS_S1/CGS_S1_SLC_L1/" + path[iw_index:]

    ana_split = stsa.TopsSplitAnalyzer(target_subswaths=['iw1', 'iw2', 'iw3'])
    ana_split.load_data(zip_path = vm_path)
    ana_split._create_subswath_geometry()
    ana_split = ana_split.df
    pyro = identify(vm_path)
    ana_split["sensor"] = pyro.sensor
    ana_split["start"] = pyro.start
    ana_split["rel_orbit"] = pyro.orbitNumber_rel
    if pyro.sensor == "S1A":
        s1a.append(ana_split)
    elif pyro.sensor == "S1B":
        s1b.append(ana_split)
    
df = gpd.GeoDataFrame(pd.concat(s1a, ignore_index = True), crs=new_list[0].crs)
dff = gpd.GeoDataFrame(pd.concat(s1b, ignore_index = True), crs=new_list[0].crs)


df.to_file("snapshot_init_oct_1-13_10_20_S1A.geojson", driver = "GeoJSON")
dff.to_file("snapshot_init_oct_1-13_10_20_S1B.geojson", driver = "GeoJSON")

[CATALOGUE]: starting search for scenes...
Loaded ZIP file: S1A_IW_SLC__1SDV_20201001T174103_20201001T174130_034606_040775_1D76.zip
Found 6 XML paths
Loaded location grid with 9 bursts and 210 coordinates
Loaded ZIP file: S1A_IW_SLC__1SDV_20201001T174128_20201001T174155_034606_040775_DB74.zip
Found 6 XML paths
Loaded location grid with 9 bursts and 210 coordinates
Loaded ZIP file: S1A_IW_SLC__1SDV_20201003T061609_20201003T061637_034628_040844_CBBF.zip
Found 6 XML paths
Loaded location grid with 10 bursts and 231 coordinates
Loaded ZIP file: S1A_IW_SLC__1SDV_20201003T061634_20201003T061701_034628_040844_0CCC.zip
Found 6 XML paths
Loaded location grid with 9 bursts and 210 coordinates
Loaded ZIP file: S1A_IW_SLC__1SDV_20201003T095707_20201003T095734_034630_04085A_F810.zip
Found 6 XML paths
Loaded location grid with 9 bursts and 210 coordinates
Loaded ZIP file: S1A_IW_SLC__1SDV_20201003T172425_20201003T172453_034635_040884_1D76.zip
Found 6 XML paths
Loaded location grid with 9 bursts and 

In [2]:
import glob
from pyroSAR import identify
import stsa
import pandas as pd
import sys, os
from contextlib import contextmanager
import geopandas as gpd

@contextmanager
def suppress_stdout():
    # source http://thesmithfam.org/blog/2012/10/25/temporarily-suppress-console-output-in-python/
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [3]:
def create_gpd_for_scene(path, id, name, all_scenes):
    """Create a geopandas dataframe for a S1 scene.
    
    Metadata is read using pyroSAR. Creates the gpd using stsa.
    A gpd with all bursts is returned.
    """
    
    # use pyroSAR to extract metadata
    pyro = identify(path)

    # use stsa to create swath geometry
    stsa_geom = stsa.TopsSplitAnalyzer(target_subswaths=['iw1', 'iw2', 'iw3'])
    with suppress_stdout():
        stsa_geom.load_data(zip_path = path)
        stsa_geom._create_subswath_geometry()

    # add attributes to bursts
    stsa_geom = stsa_geom.df
    stsa_geom["id"] = id
    stsa_geom["name"] = name
    stsa_geom["path"] = path
    stsa_geom["sensor"] = pyro.sensor
    stsa_geom["polarizations"] = ','.join(str(e) for e in pyro.polarizations)
    stsa_geom["start"] = pyro.start
    stsa_geom["stop"] = pyro.stop
    stsa_geom["mode"] = pyro.acquisition_mode
    stsa_geom["product"] = pyro.product
    stsa_geom["orbit_direction"] = pyro.orbit
    stsa_geom["rel_orbit"] = pyro.orbitNumber_rel

    # check whether the scene has a regular burst pattern (no missing, no additional bursts)
    if stsa_geom.loc[:,"burst"].to_list() == [1,2,3,4,5,6,7,8,9,1,2,3,4,5,6,7,8,9,1,2,3,4,5,6,7,8,9]:
        stsa_geom["regular_burst_pattern"] = 1
    else:
        stsa_geom["regular_burst_pattern"] = 0

    # TODO calc overlap etc.
    orbit = pyro.orbitNumber_rel
    direction = pyro.orbit
    
    all_from_orbit = all_scenes[all_scenes["rel_orbit"] == orbit]
    intersection = gpd.overlay(stsa_geom, all_from_orbit, how = "intersection")
    intersection = intersection.dissolve(["id_2", "subswath_2"], as_index = False)
    print(intersection)
    intersection.to_file("intersec.geojson", driver="GeoJSON")
    
    return(stsa_geom)

# introduce geojson
file_list = glob.glob("/data/MTDA/CGS_S1/CGS_S1_SLC_L1/IW/DV/2021/09/07/*/*.zip")

# load main dataframe
if os.path.isfile("scene_processing.geojson"):
    all_scenes = gpd.read_file("scene_processing.geojson")
    create_new_file = False
    
    # create a dataframe with a geometry for each subswath of each scene
    # all_scenes.dissolve(["id", "subswath"], as_index = False).to_file("diss_id_subs.geojson", driver = "GeoJSON")
    
else:
    print("scene_processing.geojson not found")
    create_new_file = True

new_list = []

for path in file_list:
    name = path[path.rfind("/")+1:len(path)-4] # extract product name from path
    id = name[len(name)-4:len(name)]
    
    if create_new_file:
        stsa_geom = create_gpd_for_scene(path, id, name, all_scenes)
        # append to geodataframe list
        new_list.append(stsa_geom)
    
    elif not id in all_scenes["id"].values:
        # print("found new scene")
        stsa_geom = create_gpd_for_scene(path, id, name, all_scenes)

        # append to geodataframe list
        new_list.append(stsa_geom)
        
    else:
        pass
        # print("old scene")
    
    
    
# print(set(pd.DataFrame(meta_list)["rel_orbit"].to_list()))
# orbits of the year 2021 are {1, 8, 37, 59, 88, 101, 110, 137, 139, 161} # ASF: 1, 101, 137 doesnt seem to exist over belgium
# found in 11 day baseline: {8, 37, 59, 88, 110, 139, 161}

def save_gpd(all_scenes, geodf, flag):
    """Saves a geopandas dataframe to a file.
    
    If a new file should be created, there is nothing to merge into.
    """
    
    if flag:
        geodf.to_file("scene_processing.geojson", driver = "GeoJSON")
    else:
        all_scenes = all_scenes.append(geodf, sort = False)
        all_scenes.to_file("scene_processing_all.geojson", driver = "GeoJSON")

if len(new_list) > 1:
    new_df = gpd.GeoDataFrame(pd.concat(new_list, ignore_index = True), crs=new_list[0].crs)
    save_gpd(all_scenes, new_df, create_new_file)
    print(int(len(new_df)/27), " new scenes added")
elif len(new_list) == 1:
    save_gpd(all_scenes, new_list[0], create_new_file)
    print("1 new scene added")
else:
    print("no new scenes found")
    pass # nothing needs to be saved

/usr/local/lib/python3.6/site-packages/geopandas/base.py:35: UserWarning: GeoSeries crs mismatch: EPSG:4326 and {'init': 'epsg:4326'}
  right.crs))


   id_2 subswath_2                                           geometry  \
0  7529        IW1  POLYGON ((7.603611249260235 51.26226755615214,...   
1  7529        IW2  POLYGON ((6.396449296625965 51.34839987108877,...   
2  7529        IW3  POLYGON ((5.121873303498305 51.42910844306596,...   

  subswath_1  burst_1  id_1  \
0        IW1        3  A00F   
1        IW1        4  A00F   
2        IW2        4  A00F   

                                              name_1  \
0  S1B_IW_SLC__1SDV_20210907T054938_20210907T0550...   
1  S1B_IW_SLC__1SDV_20210907T054938_20210907T0550...   
2  S1B_IW_SLC__1SDV_20210907T054938_20210907T0550...   

                                              path_1 sensor_1 polarizations_1  \
0  /data/MTDA/CGS_S1/CGS_S1_SLC_L1/IW/DV/2021/09/...      S1B           VV,VH   
1  /data/MTDA/CGS_S1/CGS_S1_SLC_L1/IW/DV/2021/09/...      S1B           VV,VH   
2  /data/MTDA/CGS_S1/CGS_S1_SLC_L1/IW/DV/2021/09/...      S1B           VV,VH   

   ...                          

In [43]:
df = gpd.read_file("scene_processing.geojson")
df = df.dissolve("id", as_index = False)
df.to_file("whole_scenes_09_1-2.geojson", driver = "GeoJSON")

hi
